## Step 1: Create a database connection and Import data


**modify the database table structure**
```
ALTER TABLE esslnc
ADD CONSTRAINT unique_lnc 
UNIQUE (chr, start, end, strand);
```

In [3]:
import pandas as pd
from sqlalchemy import create_engine,text


username = 'root'
password = 'root'
host = 'localhost'  # general 'localhost'
port = 3307  #Default MySQL port number.
database = 'dbess2'


engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')

## Step2: Import gene entries

The files needed.
1. /match/lncbook_map.tsv 
2. /match/noncode_map.tsv
3. /match/gencode_map.tsv
4. /match/non-coding_RNA.txt 
5. /match/go_map.txt
6. /match/crispr_all.bed
7. /match/merge.txt
8. /clinvar_map/db/crispr_overlap/final_lncRNA_nocrispr.bed
9. /cancer/unmap_from_dbesslnc.txt

### Step2.1 : import lncRNA verified by CRISPR 
 1. main crispr lncRNA gene
 2. update vitro column
 3. Export data for variants mapping.

In [ ]:

# 1. main crispr lncRNA gene
merge_dict = {}
with open('merge.txt', 'r') as f:
    for line in f:
        if '=' in line:
            gene1, gene2 = line.strip().split('=')
            merge_dict[gene1] = gene2
            merge_dict[gene2] = gene1

df = pd.read_csv('crispr_all.bed', sep='\t', header=None)

df['gene_id'] = df[3].apply(lambda x: x.rsplit('-', 1)[0])


merge_groups = {}
processed_genes = set()

for gene in df['gene_id'].unique():
    if gene in processed_genes:
        continue
        
    if gene in merge_dict:
        partner = merge_dict[gene]
        group_name = min(gene, partner) 
        merge_groups[gene] = group_name
        merge_groups[partner] = group_name
        processed_genes.add(gene)
        processed_genes.add(partner)
    else:
        merge_groups[gene] = gene
        processed_genes.add(gene)

df['merge_group'] = df['gene_id'].map(merge_groups)


result = df.groupby('merge_group').agg({
    0: 'first',  # chr
    1: lambda x: min(x) + 1,    # start
    2: 'max',    # end
    5: 'first',  # strand
    'gene_id': lambda x: ';'.join(sorted(set(x)))  
}).reset_index()

result = result[[0, 1, 2, 5, 'gene_id']]  
result.columns = ['chr', 'start', 'end', 'strand', 'target']  


result.to_sql('esslnc', engine, if_exists='append', index=False)
 

In [ ]:
# 2. update vitro column
with engine.connect() as conn:
    update_sql = text("""
        UPDATE esslnc 
        SET vitro = 1
    """)
    conn.execute(update_sql)
    conn.commit()

print("Update successfully")

### Step2.2:Export data for variants mapping.

In [ ]:
# 3. Export data for variants mapping.
query = """
SELECT chr,`start`,`end`,target,0,strand 
FROM esslnc
WHERE vitro = 1
"""
df = pd.read_sql(query, engine)

output_file = 'crispr.txt'
df.to_csv(output_file, sep='\t', index=False)

print(f"exported {len(df)} records {output_file}")

### Step2.3 import variants lncRNA gene to database,

In [ ]:
# 4.
# Adjust the order of column names according to different files.

input_file = 'final_lncRNA_nocrispr.bed'
df = pd.read_csv(input_file, sep='\t', header=None,
                 names=['chr', 'start', 'end', 'Lncbook_id', 'Noncode_id', 
                       'strand', 'gene_name', 'NCBI_id', 'variants_num','disease_related'])
print(len(df))

df['Lncbook_id'] = df['Lncbook_id'].replace('-', 'N.A.')
df['Noncode_id'] = df['Noncode_id'].replace('-', 'N.A.')
df['NCBI_id'] = df['NCBI_id'].replace('-', 'N.A.')

insert_data = df[['chr', 'start', 'end', 'Lncbook_id', 'Noncode_id', 
                  'strand', 'gene_name', 'NCBI_id','disease_related']]

with engine.connect() as conn:
    for _, row in insert_data.iterrows():
        try:
            insert_sql = text("""
                INSERT IGNORE INTO esslnc 
                (chr, start, end, Lncbook_id, Noncode_id, strand, gene_name, NCBI_id,disease_related)
                VALUES (:chr, :start, :end, :Lncbook_id, :Noncode_id, :strand, :gene_name, :NCBI_id,:disease_related)
            """)
            conn.execute(insert_sql, row.to_dict())
        except Exception as e:
            print(f"skip")
            continue
    conn.commit()


# try:
#     insert_data.to_sql('esslnc', engine, if_exists='append', index=False)
# except Exception as e:
#     print(f"error {str(e)}")


In [ ]:
# 5. Update disease_related column,Mark whether lncRNAs derived from CRISPR experiments are disease-related.
with engine.connect() as conn:
    update_sql = text("""
        UPDATE esslnc
        SET disease_related = 1
        WHERE vitro = 0
    """)
    conn.execute(update_sql)
    conn.commit()

print("Update successfully")

In [ ]:
# 6.unmap_from_dbesslnc.txt,After manually searching the public database GeneCards, 
# additional annotation information was supplemented.
input_file = 'unmap_from_dbesslnc.txt'
df = pd.read_csv(input_file, sep='\t')
with engine.connect() as conn:
    for _, row in df.iterrows():
        try:
            insert_sql = text("""
                INSERT IGNORE INTO esslnc 
                (chr, start, end, Lncbook_id, Noncode_id, strand, gene_name, NCBI_id,PMID)
                VALUES (:chr, :start, :end, :Lncbook_id, :Noncode_id, :strand, :gene_name, :NCBI_id, :PMID)
            """)
            conn.execute(insert_sql, row.to_dict())
        except Exception as e:
            print(f"skip")
            continue
    conn.commit()

In [34]:
# For lncRNAs from dbesslnc,mark the columns for cancer-related and in vivo.
# dbesslnc_id.txt,The file contains the gene name and the corresponding role.
df = pd.read_csv('dbesslnc_id.txt', sep='\t')

with engine.connect() as conn:
    general_genes = tuple(df[df['Role'] == 'General']['Name'].tolist())
    suppressor_genes = tuple(df[df['Role'] == 'Tumor suppressor gene']['Name'].tolist())
    oncogenes = tuple(df[df['Role'] == 'Oncogene']['Name'].tolist())

    if general_genes:
        update_general = text("""
            UPDATE esslnc 
            SET vivo = 1
            WHERE gene_name IN :genes
        """)
        conn.execute(update_general, {'genes': general_genes})

    if suppressor_genes:
        update_suppressor = text("""
            UPDATE esslnc 
            SET cancer_related = 2
            WHERE gene_name IN :genes
        """)
        conn.execute(update_suppressor, {'genes': suppressor_genes})

    if oncogenes:
        update_oncogene = text("""
            UPDATE esslnc 
            SET cancer_related = 1
            WHERE gene_name IN :genes
        """)
        conn.execute(update_oncogene, {'genes': oncogenes})

    conn.commit()

## Step3: Import the mapped gene IDs

In [8]:
# Upload the mapped database gene IDs and gene names,along with other mapped data(lncbook/noncode/gencode_map.tsv,non-coding_RNA.txt,go_map.txt)

# map_file = 'gencode_map.tsv' #*_map.tsv
map_file = 'non-coding_RNA.txt' #*.txt
# map_df = pd.read_csv(map_file, sep='\t', header=None)
map_df = pd.read_csv(map_file, sep='\t')
# map_dict = map_df[[0,4,5]].drop_duplicates()
map_dict = map_df[['symbol', 'entrez_id']]

with engine.connect() as conn:
    for _, row in map_dict.iterrows():
        # Modify column names,
        # if str(row[5]).startswith('ENSG'):
        #     continue
        # update_sql = text("""
        #     UPDATE esslnc 
        #     SET ensembl_id = :ensemnl 
        #     WHERE target LIKE :pattern1 
        #     OR target LIKE :pattern2
        # """)
        # pattern1 = f"{row[0]}%"  
        # pattern2 = f"%;{row[0]}%"  
        # conn.execute(update_sql, {
        #     "entrez_id": row[1],
        #     "pattern1": pattern1,
        #     "pattern2": pattern2
        # })
        # -------------------
        # txt
        update_sql = text("""
            UPDATE esslnc 
            SET NCBI_id = :entrez_id
            WHERE gene_name = :symbol
            
        """)
        
        
        conn.execute(update_sql, {
            "entrez_id": 'N.A.' if pd.isna(row[1]) else row[1],
            "symbol": row[0],
        })
        conn.commit()

## Step4: Generating Unique Identifiers for lncRNA Entries

### Step4.1

In [ ]:
#  Generating Unique Identifiers for lncRNA Entries
query = """
SELECT num_id, chr, start 
FROM esslnc 
ORDER BY chr, start
"""


df = pd.read_sql(query, engine)
df['new_uid'] = ['ELH{:06d}'.format(i+1) for i in range(len(df))]


with engine.connect() as conn:
    for index, row in df.iterrows():
        update_sql = text("""
        UPDATE esslnc 
        SET UID = :new_uid 
        WHERE num_id = :num_id
        """)
        conn.execute(update_sql, {"new_uid": row['new_uid'], "num_id": row['num_id']})
    conn.commit()

print(f"Successfully updated {len(df)} records。")

### Step4.2:

In [10]:
# Create a local mapping table for crispr lncRNA transcript.
query = """
SELECT UID, target 
FROM esslnc WHERE vitro = 1
"""
df = pd.read_sql(query, engine)


def split_target(row):
    targets = row['target'].split(';')
    return [{'UID': row['UID'], 'target': target} for target in targets]


expanded_rows = [item for _, row in df.iterrows() for item in split_target(row)]
result_df = pd.DataFrame(expanded_rows)


output_file = 'crispr_mapping.tsv'
result_df.to_csv(output_file, sep='\t', index=False)

print(f"exported {len(result_df)} records {output_file}")

exported 1186 records crispr_mapping.tsv


### Step4.3: 

In [11]:
# Create a local mapping table for variants lncRNA 
query = """
SELECT UID, target 
FROM esslnc WHERE vitro = 1
"""
df = pd.read_sql(query, engine)

output_file = 'crispr_UID.txt'
df.to_csv(output_file, sep='\t', index=False)

print(f"exported {len(df)} records {output_file}")

exported 1161 records crispr_UID.txt


### Step4.4:

In [ ]:
# export Create a local mapping table for disease_related lncRNA.
query = text("""
    SELECT Lncbook_id,Noncode_id,UID
    FROM esslnc 
    WHERE disease_related = 1 AND vitro = 0;
""")

output_file = 'disease_related_UID.txt'
with engine.connect() as conn:

    result = pd.read_sql(query, conn)
    

    result.to_csv(output_file, 
                  sep='\t', 
                  index=False, 
                  header=True,
                  na_rep='N.A.')

print(f"exported {len(result)} records {output_file}")

### Step4.5:

In [12]:
# export Create a local mapping table for non verified by crispr lncRNA.
query = text("""
    SELECT Lncbook_id,Noncode_id,UID
    FROM esslnc 
    WHERE  vitro = 0;
""")

output_file = 'non_crispr_UID.txt'
with engine.connect() as conn:

    result = pd.read_sql(query, conn)
    

    result.to_csv(output_file, 
                  sep='\t', 
                  index=False, 
                  header=True,
                  na_rep='N.A.')

print(f"exported {len(result)} records {output_file}")

exported 5119 records non_crispr_UID.txt


## Step5: Import transcript Entries
The required files:
1. /match/seq_splice.bed seq_delete.bed seq_casrx.bed seq_crispri.bed
2. /match/esslnc2.fa


### Step5.1

In [7]:

# import Genomic location information.
import pandas as pd

# custum bed file, generated by gen_fa.ipynb step1, Just input different files as needed
custum_bed_file = 'seq_casrx.bed'
#transcript sequence crispr_gene.fa
seq_file = 'lncRNAV2.fasta'

mapping_df = pd.read_csv('crispr_mapping.tsv', sep='\t',names=['UID','target'])
target_to_uid = {}

for _, row in mapping_df.iterrows():
    target_to_uid[row['target']] = row['UID']

df = pd.read_csv(custum_bed_file, sep='\t', header=None, 
                 names=['chr','start','end','name','score','strand','block_starts','block_sizes'])


def parse_fasta(fasta_file):
    sequences = {}
    current_seq_id = None
    current_seq = []
    
    with open(fasta_file) as f:
        for line in f:
            line = line.strip()
            if line.startswith('>'):

                if current_seq_id:
                    sequences[current_seq_id] = ''.join(current_seq)
                current_seq_id = line[1:]  
                current_seq = []
            else:
                current_seq.append(line)
    if current_seq_id:
        sequences[current_seq_id] = ''.join(current_seq)
    
    return sequences


transcript_fa = parse_fasta(seq_file)

def process_transcript(row):

    transcript_id = row['name'].rsplit('-',1)[1]
    target = row['name'].rsplit('-',1)[0]
    UID = target_to_uid.get(target)
    fasta_seq = transcript_fa.get(transcript_id)
    if fasta_seq is None:
        FASTA = f">{transcript_id}<br/>Sequence not found"
    else:
        FASTA = f">{transcript_id}<br/>{fasta_seq}"

    starts = [int(x) for x in row['block_starts'].split(',') if x]
    sizes = [int(x) for x in row['block_sizes'].split(',') if x]
    

    exon_positions = []
    for rel_start, size in zip(starts, sizes):
        abs_start = row['start'] + rel_start + 1  # 1-base
        abs_end = abs_start + size - 1
        exon_positions.append(f"{abs_start}-{abs_end}")
    
    return {
        'UID': UID,
        'transcript_id': transcript_id,
        'chr': row['chr'],
        'start': row['start'] + 1,  # 1-base
        'end': row['end'],
        'length': sum(sizes),
        'exon_num': len(sizes),
        'exon_pos': ','.join(exon_positions),
        'strand': row['strand'],
        'FASTA':FASTA
    }

result_df = pd.DataFrame([process_transcript(row) for _, row in df.iterrows()])


result_df.to_sql('trans', engine, if_exists='append', index=False)

print(f"Successfully import {len(result_df)} records to trans table.")

Successfully import 4635 records to trans table.


### Step5.2:

In [ ]:
# import non verified by crispr lncRNA 

import pandas as pd
from sqlalchemy import text

non_crispr_file = 'non_crispr_lncrna.csv'
headers = pd.read_csv(non_crispr_file, nrows=0).columns.tolist()
print(headers)

sql = text(f"""
    INSERT IGNORE INTO trans
    ({', '.join(headers)})
    VALUES ({', '.join([':' + col for col in headers])})
""")

total_rows = 0
success_rows = 0

for chunk in pd.read_csv(non_crispr_file, chunksize=1000):
    with engine.connect() as conn:
        for _, row in chunk.iterrows():
            try:
                result = conn.execute(sql, row.to_dict())
                success_rows += result.rowcount
                total_rows += 1
            except Exception as e:
                print(e)
                continue
        conn.commit()
print(f"Total processed: {total_rows}")
print(f"Successfully imported: {success_rows}")
print(f"Skipped duplicates: {total_rows - success_rows}")

['Lncbook_id', 'NONCODE_Gene_ID', 'UID', 'NONCODE_TRANSCRIPT_ID', 'Lncbook_trans_id', 'chr', 'start', 'end', 'strand', 'exon_num', 'exon_pos', 'transcript_id']
Total processed: 29772
Successfully imported: 29772
Skipped duplicates: 0


### Step5.3 Import sequence (lncRNA non verified by crispr)
The required files(downloaded from lncbook v2.0 and Noncode V6.0)
1. LncBookv2_OnlyLnc.fa
2. outLncRNA.fa 


In [ ]:
# import seq from lncbook/noncode
import pandas as pd
from sqlalchemy import text

fa_file = 'LncBookv2_OnlyLnc.fa'

sql_select = text("""
    SELECT transcript_id 
    FROM trans 
    WHERE FASTA IS NULL
""")


fasta_dict = {}
with open(fa_file, 'r') as f:
    seq = ''
    tid = ''
    for line in f:
        if line.startswith('>'):
            if tid and seq:
                fasta_dict[tid] = seq
            tid = line.strip().lstrip('>')
            seq = ''
        else:
            seq += line.strip()
    if tid and seq:
        fasta_dict[tid] = seq


sql_update = text("""
    UPDATE trans 
    SET FASTA = :fasta 
    WHERE transcript_id = :tid
""")


with engine.connect() as conn:
    transcript_ids = conn.execute(sql_select).fetchall()

    for (tid,) in transcript_ids:
        if tid in fasta_dict:
            fasta_content = f"{tid}<br/>{fasta_dict[tid]}"
            try:
                conn.execute(sql_update, {"fasta": fasta_content, "tid": tid})
            except Exception as e:
                print(f"Error updating {tid}: {e}")
    
    conn.commit()


## Step6: Import the mapped transcript IDs
The required files
1. /match/lncbook_map.tsv
2. /match/noncode_map.tsv

In [8]:
# Upload the mapped database gene IDs and transcript IDs,along with other mapped data.
# You can modify SQL statements and input files to update fields in the database.
map_file = 'lncbook_map.tsv' #*_map.tsv,

map_df = pd.read_csv(map_file, sep='\t', header=None)
# filter
map_dict = map_df[map_df[6] == 'transcript'][[1,3,4]].drop_duplicates()
map_dict.columns = ['transcript_id','Lncbook_trans_id', 'Lncbook_id'] # You can modify the corresponding column names.
# print(map_dict.head())


for _, lnc_row in map_dict.iterrows():
    try:
        with engine.connect() as conn:
            update_sql = text("""
            UPDATE trans 
            SET Lncbook_trans_id = :Lncbook_trans_id, 
                Lncbook_id = :Lncbook_id
            WHERE transcript_id = :transcript_id
            """)
            
            result = conn.execute(update_sql, {
                "transcript_id": lnc_row['transcript_id'],
                "Lncbook_id": lnc_row['Lncbook_id'],
                "Lncbook_trans_id": lnc_row['Lncbook_trans_id']
            })
            
            rows_affected = result.rowcount
            # print(f"update records: {lnc_row['transcript_id']}, affected rows: {rows_affected}")
            
            conn.commit()
    except Exception as e:
        print(f"update fail: {lnc_row['transcript_id']}, error: {str(e)}")

## Step7: import CRISPR experiment record `/curated/exp_crispr.csv`

In [ ]:

exp_df = pd.read_csv('../store/exp_crispr.csv')

exp_df.to_sql('exp_crispr', 
              engine, 
              if_exists='append', 
              index=False,         
              chunksize=1000)      

print(f"Successfully import {len(exp_df)} records to trans table.")

### step7.1 
Group by   exp_type   and   target_id   columns, for groups with a count of 1, mark as 'cell-line specific' and update in the table; for groups with a count of 2-5, mark as 'common essential'; and for groups with a count greater than 5, mark as 'core essential'.

In [12]:
import pandas as pd
from sqlalchemy import text


sql_select = text("""
    SELECT exp_type, target_id 
    FROM exp_crispr
""")

with engine.connect() as conn:

    df = pd.read_sql(sql_select, conn)
    

    group_counts = df.groupby(['exp_type', 'target_id']).size()
    

    cell_specific = group_counts[group_counts == 1].index
    common = group_counts[(group_counts >= 2) & (group_counts <= 5)].index
    core = group_counts[group_counts > 5].index
    

    update_sql = text("""
        UPDATE exp_crispr
        SET role = :etype 
        WHERE exp_type = :exp AND target_id = :tid
    """)
    

    for exp, tid in cell_specific:
        conn.execute(update_sql, {"etype": "cell-line specific", "exp": exp, "tid": tid})
        
    for exp, tid in common:
        conn.execute(update_sql, {"etype": "common essential", "exp": exp, "tid": tid})
        
    for exp, tid in core:
        conn.execute(update_sql, {"etype": "core essential", "exp": exp, "tid": tid})
    
    conn.commit()

## Step8：import variants Mapping table and variants
The required files.
1. /clinvar_map/db/construct_map/disease_mapping.csv
2. /clinvar_map/db/construct_map/crispr_mapping.csv
3. /clinvar_map/db/crispr_overlap/variants_nocrispr.csv
4. /clinvar_map/db/crispr_map/crispr_variants.csv


In [ ]:
# Import variant information and mapping tables into the database.
import pandas as pd
from sqlalchemy import text

variants_file = 'disease_mapping.csv'
headers = pd.read_csv(variants_file, nrows=0).columns.tolist()

#table name variants/lncrna_variant_mapping
sql = text(f"""
    INSERT IGNORE INTO lncrna_variant_mapping
    ({', '.join(headers)})
    VALUES ({', '.join([':' + col for col in headers])})
""")

total_rows = 0
success_rows = 0

for chunk in pd.read_csv(variants_file, chunksize=1000):
    with engine.connect() as conn:
        for _, row in chunk.iterrows():
            try:
                result = conn.execute(sql, row.to_dict())
                success_rows += result.rowcount
                total_rows += 1
            except Exception as e:
                continue
        conn.commit()
print(f"Total processed: {total_rows}")
print(f"Successfully imported: {success_rows}")
print(f"Skipped duplicates: {total_rows - success_rows}")


## Step9: import expression profile
The required file
1. /clinvar_map/db/exp/exp_profile.csv

In [11]:
# import all lncRNA expression profile

import pandas as pd
from sqlalchemy import text

expression_file = 'exp_profile.csv'
headers = pd.read_csv(expression_file, nrows=0).columns.tolist()
print(headers)

sql = text(f"""
    INSERT IGNORE INTO exp_profile
    ({', '.join(headers)})
    VALUES ({', '.join([':' + col for col in headers])})
""")

total_rows = 0
success_rows = 0

for chunk in pd.read_csv(expression_file, chunksize=1000):
    with engine.connect() as conn:
        for _, row in chunk.iterrows():
            try:
                result = conn.execute(sql, row.to_dict())
                success_rows += result.rowcount
                total_rows += 1
            except Exception as e:
                print(e)
                continue
        conn.commit()
print(f"Total processed: {total_rows}")
print(f"Successfully imported: {success_rows}")
print(f"Skipped duplicates: {total_rows - success_rows}")

['UID', 'Lncbook_trans_id', 'transcript_id', 'brain', 'lung', 'urinarybladder', 'kidney', 'adrenal', 'thyroid', 'heart', 'lymphnode', 'spleen', 'bonemarrow', 'tonsil', 'appendix', 'colon', 'esophagus', 'gallbladder', 'smallintestine', 'salivarygland', 'stomach', 'liver', 'duodenum', 'pancreas', 'rectum', 'endometrium', 'ovary', 'testis', 'prostate', 'fallopiantube', 'skeletalmuscle', 'smoothmuscle', 'skin', 'fat']
Total processed: 26642
Successfully imported: 26642
Skipped duplicates: 0
